In [2]:
import pandas as pd

In [3]:
%store -r PATH_TO_DATA_FOLDER

data = pd.read_csv(PATH_TO_DATA_FOLDER+"/data_chunk_1.csv")
#generalise this file name

In [5]:
print((data.lat.describe()))
print((data.long.describe()))
#clearly there are zero values and values very far from the mean
#these must be device errors since the bus can't deviate by 80 units
#these values are hence dropped

count    400.000000
mean      13.121761
std        5.695504
min        0.000000
25%       12.919502
50%       12.966616
75%       13.018760
max      100.000000
Name: lat, dtype: float64
count     400.000000
mean       80.554366
std        62.058616
min         0.000000
25%        77.536640
50%        77.585838
75%        77.634634
max      1000.000000
Name: long, dtype: float64


In [8]:
#remove rows which have 0 lat values(by picking a subset of rows that don't have a zero lat value)
lat_nullfree = data[data.lat != 0.0]

#now remove the rows which have 0 long values
lat_long_nullfree = data[data.long != 0.0]

#min value is no longer zero
print(lat_long_nullfree.describe())

             bus_id         lat         long
count  3.930000e+02  393.000000   393.000000
mean   1.504179e+08   13.355482    81.989176
std    2.794590e+05    5.467048    61.661215
min    1.502181e+08   12.680088    77.394203
25%    1.502201e+08   12.920876    77.537109
50%    1.502218e+08   12.976646    77.586082
75%    1.508114e+08   13.018923    77.634689
max    1.508148e+08  100.000000  1000.000000


In [9]:
#collect mean and define a safe error margin
lat_mean = lat_long_nullfree.mean()["lat"]
long_mean = lat_long_nullfree.mean()["long"]

lat_error = 10
long_error = 10

In [11]:
#remove rows where the mean value and actual value differ by more than the error

#first for lat
lat_proper = lat_long_nullfree[abs(lat_long_nullfree.lat - lat_mean) < lat_error]

#now for long on top of this
lat_long_proper = lat_proper[abs(lat_proper.long - long_mean) < long_error]

print(lat_long_proper.describe())

#the max value is now well within the error, so it was safe
#using the fact that the lat, long deviation of buses can't be very high


             bus_id         lat        long
count  3.910000e+02  391.000000  391.000000
mean   1.504174e+08   12.969171   77.595080
std    2.792901e+05    0.086997    0.082799
min    1.502181e+08   12.680088   77.394203
25%    1.502201e+08   12.920704   77.537064
50%    1.502218e+08   12.974773   77.585938
75%    1.508114e+08   13.018783   77.634640
max    1.508148e+08   13.300202   77.862457


In [12]:
#now write that data into a csv file

lat_long_proper.to_csv(PATH_TO_DATA_FOLDER + "/cleaned_data_chunk_1.csv",columns = ['bus_id', 'lat', 'long', 'time'],index=None)